In [1]:
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageEnhance
import face_recognition
import numpy as np


class FaceDetectionTool:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Detection Enhancement Tool")
        self.image_path = None
        self.original_image = None
        self.processed_image = None

        # Default enhancement parameters
        self.brightness = 1.0
        self.sharpness = 1.0
        self.contrast = 1.0

        # Create GUI elements
        self.create_gui()

    def create_gui(self):
        # Frame for image display
        self.image_frame = tk.Label(self.root)
        self.image_frame.grid(row=0, column=0, columnspan=3, padx=10, pady=10)

        # Buttons to load and process images
        tk.Button(self.root, text="Load Image", command=self.load_image).grid(row=1, column=0, padx=5, pady=5)
        tk.Button(self.root, text="Check Faces", command=self.check_faces).grid(row=1, column=1, padx=5, pady=5)
        tk.Button(self.root, text="Save Processed Image", command=self.save_image).grid(row=1, column=2, padx=5, pady=5)

        # Sliders for adjustments
        tk.Label(self.root, text="Brightness").grid(row=2, column=0)
        self.brightness_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_brightness
        )
        self.brightness_slider.set(1.0)
        self.brightness_slider.grid(row=2, column=1, columnspan=2)

        tk.Label(self.root, text="Sharpness").grid(row=3, column=0)
        self.sharpness_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_sharpness
        )
        self.sharpness_slider.set(1.0)
        self.sharpness_slider.grid(row=3, column=1, columnspan=2)

        tk.Label(self.root, text="Contrast").grid(row=4, column=0)
        self.contrast_slider = tk.Scale(
            self.root, from_=0.5, to=2.0, resolution=0.1, orient="horizontal", command=self.update_contrast
        )
        self.contrast_slider.set(1.0)
        self.contrast_slider.grid(row=4, column=1, columnspan=2)

    def load_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.png;*.jpeg")])
        if not file_path:
            return
        self.image_path = file_path
        self.original_image = cv2.cvtColor(cv2.imread(self.image_path), cv2.COLOR_BGR2RGB)
        self.processed_image = self.original_image.copy()
        self.display_image(self.original_image)

    def display_image(self, image):
        img = Image.fromarray(image)
        imgtk = ImageTk.PhotoImage(img)
        self.image_frame.imgtk = imgtk
        self.image_frame.configure(image=imgtk)

    def apply_enhancements(self):
        if self.original_image is None:
            return
        pil_image = Image.fromarray(self.original_image)
        pil_image = ImageEnhance.Brightness(pil_image).enhance(self.brightness)
        pil_image = ImageEnhance.Sharpness(pil_image).enhance(self.sharpness)
        pil_image = ImageEnhance.Contrast(pil_image).enhance(self.contrast)
        self.processed_image = np.array(pil_image)
        self.display_image(self.processed_image)

    def update_brightness(self, value):
        self.brightness = float(value)
        self.apply_enhancements()

    def update_sharpness(self, value):
        self.sharpness = float(value)
        self.apply_enhancements()

    def update_contrast(self, value):
        self.contrast = float(value)
        self.apply_enhancements()

    def check_faces(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No image loaded.")
            return
        # Detect faces
        face_locations = face_recognition.face_locations(self.processed_image)
        num_faces = len(face_locations)
        if num_faces > 0:
            for top, right, bottom, left in face_locations:
                cv2.rectangle(
                    self.processed_image, (left, top), (right, bottom), (0, 255, 0), 2
                )
            self.display_image(self.processed_image)
            messagebox.showinfo("Faces Detected", f"Number of faces detected: {num_faces}")
        else:
            messagebox.showinfo("No Faces Detected", "No faces detected in the image.")

    def save_image(self):
        if self.processed_image is None:
            messagebox.showerror("Error", "No processed image to save.")
            return
        save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png")])
        if save_path:
            cv2.imwrite(save_path, cv2.cvtColor(self.processed_image, cv2.COLOR_RGB2BGR))
            messagebox.showinfo("Image Saved", f"Image saved to {save_path}")


if __name__ == "__main__":
    root = tk.Tk()
    app = FaceDetectionTool(root)
    root.mainloop()
